## Chord progression maker

- This notebook uses my Piano Triads dataset. -> https://www.kaggle.com/davidbroberts/piano-triads-wavset 
- My idea here is to create a list of chords that belong in each key and auto-generate progressions for playback, then string together wavs to make a song.
- The chords were recorded as whole notes at 120 BPM.
- Right now, it's random driven, but I think it needs an AI component to create style or genre based progressions.
- Of course it would be easier done with MIDI, but I thought it might be cool to train a CNN on the wavs and predict some 'feelings' into the mix.
- The other notebook I made on this dataset ( https://www.kaggle.com/davidbroberts/piano-triads-wavset2 ), shows how to load and play chords, and display the notes on a piano keyboard.

In [ ]:
import numpy as np
import pandas as pd
from scipy.io.wavfile import read
from IPython.display import Audio
import math
import random

In [ ]:
# Load the note data, these are the notes that make up each chord. They're not used here, they're just for reference.
triads = pd.read_csv("/kaggle/input/piano-triads-wavset/triads.csv")
triads.head()

In [ ]:
# Make a dataframe of all the chords in each major and minor key
keys = pd.DataFrame([
    ['C_maj','D_min','E_min','F_maj','G_maj','A_min','Bb_min'],
    ['Cs_maj','Eb_min','F_min','Fs_maj','Gs_maj','Bb_min','C_min'],
    ['D_maj','E_min','Fs_min','G_maj','A_maj','B_min','Cs_min'],
    ['Eb_maj','F_min','G_min','Gs_maj','Bb_maj','C_min','D_min'],
    ['E_maj','Fs_min','Gs_min','A_maj','C_maj','Cs_min','Eb_min'],
    ['F_maj','G_min','A_min','Bb_maj','Cs_maj','D_min','E_min'],
    ['Fs_maj','Gs_min','Bb_min','C_maj','D_maj','Eb_min','F_min'],
    ['G_maj','A_min','C_min','Cs_maj','Eb_maj','E_min','Fs_min'],
    ['Gs_maj','Bb_min','Cs_min','D_maj','E_maj','F_min','G_min'],
    ['A_maj','B_min','D_min','Eb_maj','F_maj','Fs_min','Gs_min'],
    ['Bb_maj','C_min','Eb_min','E_maj','Fs_maj','G_min','A_min'],
    ['B_maj','Cs_min','E_min','F_maj','G_maj','Gs_min','Bb_min'],    
    ['C_min','D_dim','Eb_maj','F_min','G_min','Gs_maj','Bb_maj'],
    ['Cs_min','Eb_dim','E_maj','Fs_min','Gs_min','A_maj','B_maj'],
    ['D_min','E_dim','F_maj','G_min','A_min','Bb_maj','C_maj'],
    ['Ds_min','F_dim','Fs_maj','Gs_min','Bb_min','C_maj','Cs_maj'],
    ['E_min','Fs_dim','G_maj','A_min','B_min','Cs_maj','D_maj'],
    ['F_min','G_dim','Gs_maj','Bb_min','C_min','D_maj','Ds_maj'],
    ['Fs_min','Gs_dim','A_maj','B_min','Cs_min','Ds_maj','E_maj'],
    ['G_min','A_dim','Bb_maj','C_min','D_min','E_maj','F_maj'],
    ['Gs_min','Bb_dim','C_maj','Cs_min','Ds_min','F_maj','Fs_maj'],
    ['A_min','B_dim','Cs_maj','D_min','E_min','Fs_maj','G_maj'],
    ['Bb_min','C_dim','D_maj','Ds_min','F_min','G_maj','Gs_maj'],
    ['B_min','Cs_dim','Ds_maj','E_min','Fs_min','Gs_maj','A_maj']    
])

# Take a look at the chord/key data
keys.head(12)

In [ ]:
# This function randomly chooses the chords from a given key
def get_chords(key, num_chords, octave):
    all_chords = keys.loc[keys[0] == key]
    chords = []
    
    # Iterate through the key/chord array
    for i in range(0,num_chords):
        
        # Pick a random chord from the list
        chord_index = random.randint(0,6)
        
        # Pick a random inversion
        inversion = random.randint(0,1)
        
        # Add it to the array
        chords.append(all_chords[chord_index].values[0] + "_" + str(octave) + "_" + str(inversion))
        
    return chords

### Make a verse

In [ ]:
verse = []

# Generate 4 random chords in the key of C Major at the fifth octave for the verse
verse_iterations = 2
verse_chords = get_chords('C_maj', 4, 5)
print ("Verse chords: " + str(verse_chords))

# Iterate over the verse wavs and concat them to make a verse
for c in range(0,len(verse_chords)):
    sample_rate, wav = read("../input/piano-triads-wavset/piano_triads/" + verse_chords[c] + ".wav")
    verse = np.concatenate((verse, wav), axis=0)

# Repeat the pattern 'verse_iterations' times
for i in range(1, verse_iterations):
    verse = np.concatenate((verse, verse), axis=0)
    
Audio(verse, rate=sample_rate)

- We get 24 seconds of audio because each wav is seconds long .. there are four wavs in the verse and we repeat it twice.
- 3 x 4 x 2

### Make a chorus

In [ ]:
chorus = []

# Generate 3 random chords in the key of C minor at the fourth octave for the chorus
chorus_iterations = 2
chorus_chords = get_chords('C_min', 3, 4)
print ("Chorus chords: " + str(chorus_chords))

# Iterate over the chorus wavs and concat them to make a chorus
for c in range(0,len(chorus_chords)):
    sample_rate, wav = read("../input/piano-triads-wavset/piano_triads/" + chorus_chords[c] + ".wav")
    chorus = np.concatenate((chorus, wav), axis=0)

# Repeat the pattern 'chorus_iterations' times
for i in range(1, chorus_iterations):
    chorus = np.concatenate((chorus, chorus), axis=0)

Audio(chorus, rate=sample_rate)

### Put the song together
- String together the parts to make a complete song.

In [ ]:
song = np.concatenate((verse, verse, chorus, verse), axis=0)

# Display a player
Audio(song, rate=sample_rate)